In [10]:
from langchain_community.document_loaders import PyPDFLoader

document = PyPDFLoader("rag.pdf")

loader = document.lazy_load()

text = ""

for page in loader:
    text += page.page_content

In [11]:
from langchain_text_splitters import CharacterTextSplitter


text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=100,
    separator="\n" 
)

texts = text_splitter.create_documents([text])

In [12]:
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(model="all-minilm")



In [13]:
from langchain_community.vectorstores import FAISS

# Crear el índice FAISS desde los documentos
vector_store = FAISS.from_documents(texts, embedding)

# Guardarlo en disco
vector_store.save_local("./vectorStore-pdf-faiss")

In [14]:
vector_store = FAISS.load_local(
    "./vectorStore-pdf-faiss",
    embeddings=embedding,
    allow_dangerous_deserialization=True
)

In [15]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOllama(model="llama3", temperature=0.5)


In [ ]:

ai_msg = [""]
human_msg = []

prompt_template = ChatPromptTemplate([
    ("system", """
Eres un asistente especializado en LangChain con acceso a documentación técnica actualizada.

INSTRUCCIONES ESPECÍFICAS:
1. **Análisis del contexto**: Examina cuidadosamente el {context} proporcionado
2. **Atribución de fuentes**: Cuando uses información del contexto, menciona específicamente de qué parte proviene
3. **Honestidad intelectual**: Si el contexto no contiene información suficiente, indica claramente "No tengo suficiente información para responder esa pregunta basándome en el contexto proporcionado"
4. **Estructura de respuesta**:
   - Comienza con un resumen conciso
   - Proporciona detalles técnicos relevantes
   - Incluye ejemplos prácticos cuando sea posible
   - Menciona casos de uso aplicables
5. **Evitar especulación**: No inventes características o funcionalidades no presentes en el contexto



Contexto disponible:
{context}
""")
])

In [17]:
input_user = input("Human: ")
human_msg.append(input_user)

results = vector_store.similarity_search(input_user, k=3)
context = "\n\n".join([doc.page_content for doc in results])

In [19]:
# 8. Invocar modelo
prompt = prompt_template.invoke({
    "context": results,
    "ai_msg": ai_msg,
    "human_msg": human_msg
})

response = llm.invoke(prompt)
print(response.content)

La tecnología RAG (generación aumentada por recuperación) aporta varios beneficios a los esfuerzos de la IA generativa de una organización.

 ● Implementación rentable: El desarrollo de chatbots normalmente comienza con un modelo fundamental. Los modelos fundamentales (FM) son LLM accesibles para API entrenados en un amplio espectro de datos generalizados y sin etiquetar. Los costos computacionales y financieros de volver a entrenar los FM para obtener información específica de la organización son altos. La RAG es un enfoque más rentable para introducir nuevos datos en el LLM.

 ● Mayor control para los desarrolladores: Con la RAG, los desarrolladores pueden probar y mejorar sus aplicaciones de chat de manera más eficiente. Pueden controlar y cambiar las fuentes de información del LLM para adaptarse a los requisitos cambiantes y multifuncionales. Los desarrolladores también pueden restringir la recuperación de información confidencial a diferentes niveles de autorización y garantizar q